In [2]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, LineByLineTextDataset


In [3]:
# tokenizer = ByteLevelBPETokenizer()


paths = './data/vocab.txt'
# tokenizer = BertWordPieceTokenizer(paths, lowercase=True)
tokenizer = BertTokenizer.from_pretrained(paths)


# # Customize training
# tokenizer.train(vocab_file=paths, vocab_size=82, min_frequency=2)

# Save files to disk
tokenizer.save_model(".", "CCF")

/root/miniconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1656: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


AttributeError: 'BertTokenizer' object has no attribute 'save_model'

In [3]:
%%time
dataset= LineByLineTextDataset(
    tokenizer = tokenizer,
#     file_path = './data/LineText_for_BERT.txt',
    file_path = './data/LineText_for_BERT_debug.txt',
    block_size = 512  # maximum sequence length
)

print('No. of lines: ', len(dataset)) # No of lines in your datset

/root/miniconda3/lib/python3.8/site-packages/transformers/data/datasets/language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


No. of lines:  181813
CPU times: user 1min 29s, sys: 736 ms, total: 1min 29s
Wall time: 3min 20s


In [4]:
from transformers import BertConfig, BertForMaskedLM, DataCollatorForLanguageModeling
config = BertConfig(
    vocab_size=28198,
    hidden_size=768, 
    num_hidden_layers=6, 
    num_attention_heads=12,
    max_position_embeddings=512
)
 
model = BertForMaskedLM(config)
print('No of parameters: ', model.num_parameters())

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

No of parameters:  65199910


In [5]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./pretrain_model',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    save_steps=10000,
    save_total_limit=2,
    prediction_loss_only=True,
#     no_cuda = True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

In [ ]:
%%time
trainer.train()
trainer.save_model('./pretrain_model')

/root/miniconda3/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 181813
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 11364
